In [1]:
import torchvision
import torch
import torch.nn as nn
import sys
sys.path.append("../misc");
from utils import DataShaper
import os
import math
import matplotlib.pyplot as plt
import numpy as np
from brevitas.nn import QuantConv2d, QuantIdentity, QuantReLU
from brevitas.quant.fixed_point import Int8ActPerTensorFixedPoint, Int8WeightPerTensorFixedPoint,Uint8ActPerTensorFixedPoint

torch.manual_seed(0)
design="layer_1"
# aie_teardown()
sys.path.append("../../../utils"); import xrtutils
xclbin_path = os.path.abspath("../network/"+design+"/build/final.xclbin")
insts_path  = os.path.abspath("../network/"+design+"/build/insts.txt")

log_folder="log/log_"+design

enable_aie = True
aie_is_setup = False
enable_trace = False
trace_file='traces/trace.txt'

app = None
in_buf = None
arg1_buf = None
out_buf = None
dtype_in  = np.dtype("int8")
dtype_wts  = np.dtype("int8")
dtype_out = np.dtype("uint8")

shape_in_act   = (32,8,32,8)  #'YCXC8' , 'CYX'
# shape_in_wts1  = (8,8,1,1,8,8) #out,in,ky,kx,in8,out8
# shape_in_wts2  = (8,8,3,3,8,8)  #out,in,ky,kx,in8,out8
# shape_in_wts3  = (32,8,1,1,8,8)   #out,in,ky,kx,in8,out8
shape_in_wts_skip  = (8,32,1,1,8,8) #out,in,ky,kx,in8,out8
shape_total_wts= (143360,1)
shape_out      = (32,32,32,8)

trace_size = 8192

def setup_aie(xclbin_path, insts_path, 
              in_0_shape, in_0_dtype,
              in_1_shape, in_1_dtype, 
              out_buf_shape, out_buf_dtype,
              enable_trace=False,
              kernel_name="MLIR_AIE"):
    app = xrtutils.AIE_Application(xclbin_path, insts_path, kernel_name)
    app.register_buffer(2, shape=in_0_shape, dtype=in_0_dtype)
    app.register_buffer(3, shape=in_1_shape, dtype=in_1_dtype)
    if enable_trace:
      out_buf_len_bytes = np.prod(out_buf_shape) * np.dtype(out_buf_dtype).itemsize
      out_buf_shape = (out_buf_len_bytes + trace_size, )
      out_buf_dtype = np.uint8
    app.register_buffer(4, shape=out_buf_shape, dtype=out_buf_dtype)
    return app

def extract_trace(out_buf, out_buf_shape, out_buf_dtype):
    trace_size_words = trace_size//4
    out_buf_flat = out_buf.reshape((-1,)).view(np.uint32)
    output_prefix = out_buf_flat[:-trace_size_words].view(out_buf_dtype).reshape(out_buf_shape)
    trace_suffix = out_buf_flat[-trace_size_words:]
    return output_prefix, trace_suffix

def write_out_trace(trace, file_name):
    out_str = "\n".join(f"{i:0{8}x}" 
                        for i in trace
                        if i != 0)
    with open(file_name, 'w') as f:
      f.write(out_str)

app = setup_aie(xclbin_path, insts_path,
                            shape_in_act, dtype_in,      
                            shape_total_wts,dtype_wts,
                            shape_out, dtype_out,enable_trace)

import matplotlib.pyplot as plt
import numpy as np
from brevitas.nn import QuantConv2d, QuantIdentity, QuantReLU
from brevitas.quant.fixed_point import Int8ActPerTensorFixedPoint, Int8WeightPerTensorFixedPoint,Uint8ActPerTensorFixedPoint

torch.manual_seed(0)
torch.use_deterministic_algorithms(True)

if not os.path.exists(log_folder):
    os.makedirs(log_folder)

input=torch.randn(1, 64,32,32)
num_classes=10
ds = DataShaper()
def init_pad_input(x,input_channels,desired_channels=4):
           padding=torch.zeros(1,input_channels*(desired_channels-1),32,32)
           return torch.cat((x,padding),1)
# try:
for i in range (0,1): 
    class QuantBottleneck_projected(nn.Module):
        expansion = 4
        def __init__(self, in_planes=64, planes=64):
            super(QuantBottleneck_projected, self).__init__()
            # block 0
            self.quant_id_1 = QuantIdentity(act_quant=Int8ActPerTensorFixedPoint,bit_width=8, return_quant_tensor=True) 
            self.quant_block0_conv1 = QuantConv2d(in_planes, planes, kernel_size=1,bit_width=8,weight_bit_width=8, bias=False, weight_quant=Int8WeightPerTensorFixedPoint, return_quant_tensor=True)
            self.quant_block0_conv2 = QuantConv2d(planes, planes, kernel_size=3,bit_width=8,weight_bit_width=8, bias=False,padding=1,padding_mode ='zeros', weight_quant=Int8WeightPerTensorFixedPoint, return_quant_tensor=True)
            self.quant_block0_conv3 = QuantConv2d(planes, self.expansion *planes, kernel_size=1,bit_width=8,weight_bit_width=8, bias=False,weight_quant=Int8WeightPerTensorFixedPoint, return_quant_tensor=True)
            self.quant_block0_relu1 = QuantReLU(act_quant=Uint8ActPerTensorFixedPoint ,bit_width=8, return_quant_tensor=True)
            self.quant_block0_relu2 = QuantReLU(act_quant=Uint8ActPerTensorFixedPoint ,bit_width=8, return_quant_tensor=True)
            self.quant_block0_relu3 = QuantReLU(act_quant=Uint8ActPerTensorFixedPoint ,bit_width=8, return_quant_tensor=True)
            
            self.shortcut = QuantConv2d(in_planes, self.expansion*planes, kernel_size=1,bit_width=8,weight_bit_width=8, bias=False, weight_quant=Int8WeightPerTensorFixedPoint, return_quant_tensor=True)
            
            # block 1
            self.quant_block1_conv1 = QuantConv2d(self.expansion *in_planes, planes, kernel_size=1,bit_width=8,weight_bit_width=8, bias=False, weight_quant=Int8WeightPerTensorFixedPoint, return_quant_tensor=True)
            self.quant_block1_conv2 = QuantConv2d(planes, planes, kernel_size=3,bit_width=8,weight_bit_width=8, bias=False,padding=1,padding_mode ='zeros', weight_quant=Int8WeightPerTensorFixedPoint, return_quant_tensor=True)
            self.quant_block1_conv3 = QuantConv2d(planes, self.expansion *planes, kernel_size=1,bit_width=8,weight_bit_width=8, bias=False,weight_quant=Int8WeightPerTensorFixedPoint, return_quant_tensor=True)
            self.quant_block1_relu1 = QuantReLU(act_quant=Uint8ActPerTensorFixedPoint ,bit_width=8, return_quant_tensor=True)
            self.quant_block1_relu2 = QuantReLU(act_quant=Uint8ActPerTensorFixedPoint ,bit_width=8, return_quant_tensor=True)
            self.quant_block1_relu3 = QuantReLU(act_quant=Uint8ActPerTensorFixedPoint ,bit_width=8, return_quant_tensor=True)
            
            self.quant_add_1 = QuantIdentity(act_quant=Int8ActPerTensorFixedPoint ,bit_width=8, return_quant_tensor=True)
            # Quant_add_1 shares the scale factors with block0_relu3, however one is signed and the other one is unsigned
            self.quant_add_1.act_quant.fused_activation_quant_proxy.tensor_quant.scaling_impl = self.quant_block0_relu3.act_quant.fused_activation_quant_proxy.tensor_quant.scaling_impl
            self.quant_add_1.act_quant.fused_activation_quant_proxy.tensor_quant.int_scaling_impl = self.quant_block0_relu3.act_quant.fused_activation_quant_proxy.tensor_quant.int_scaling_impl
           
        
        def forward(self, x):
            out_q = self.quant_id_1(x)
            out_rhs = self.quant_block0_conv1(out_q)
            out_rhs = self.quant_block0_relu1(out_rhs)
            out_rhs = self.quant_block0_conv2(out_rhs)
            out_rhs = self.quant_block0_relu2(out_rhs)
            out_rhs = self.quant_block0_conv3(out_rhs)
            out_rhs = self.quant_id_1(out_rhs)
            out_lhs=self.shortcut(out_q)
            out_lhs = self.quant_id_1(out_lhs)
            out_block0 = out_rhs+out_lhs
            out_block0 = self.quant_block0_relu3(out_block0)    
            # block 1
            out_rhs1 = self.quant_block1_conv1(out_block0)
            out_rhs1 = self.quant_block1_relu1(out_rhs1)
            out_rhs1 = self.quant_block1_conv2(out_rhs1)
            out_rhs1 = self.quant_block1_relu2(out_rhs1)
            out_rhs1 = self.quant_block1_conv3(out_rhs1)
            out_rhs1 = self.quant_add_1(out_rhs1)
            out_block1=out_block0+out_rhs1
            out_block1 = self.quant_block1_relu3(out_block1)
            
            return out_block1
                

    
    quant_bottleneck_model=QuantBottleneck_projected()
    
    quant_id_1 = QuantIdentity(act_quant=Int8ActPerTensorFixedPoint,bit_width=8, return_quant_tensor=True) 
    quant_bottleneck_model.eval()
    quant_id_1.eval()
  
    init_scale = quant_bottleneck_model.quant_id_1.quant_act_scale()
    block_0_relu_1 = quant_bottleneck_model.quant_block0_relu1.quant_act_scale()
    block_0_relu_2= quant_bottleneck_model.quant_block0_relu2.quant_act_scale()
    block_0_relu_3= quant_bottleneck_model.quant_block0_relu3.quant_act_scale()
    
    block_0_weight_scale1 = quant_bottleneck_model.quant_block0_conv1.quant_weight_scale()
    block_0_weight_scale2 = quant_bottleneck_model.quant_block0_conv2.quant_weight_scale()
    block_0_weight_scale3 = quant_bottleneck_model.quant_block0_conv3.quant_weight_scale()
    block_0_weight_scale_skip = quant_bottleneck_model.shortcut.quant_weight_scale()
    

    # Block 2
    block_1_relu_1 = quant_bottleneck_model.quant_block1_relu1.quant_act_scale()
    block_1_relu_2= quant_bottleneck_model.quant_block1_relu2.quant_act_scale()
    block_1_relu_3= quant_bottleneck_model.quant_block1_relu3.quant_act_scale()
    
    block_1_weight_scale1 = quant_bottleneck_model.quant_block1_conv1.quant_weight_scale()
    block_1_weight_scale2 = quant_bottleneck_model.quant_block1_conv2.quant_weight_scale()
    block_1_weight_scale3 = quant_bottleneck_model.quant_block1_conv3.quant_weight_scale()
    block_1_quant_add_1 = quant_bottleneck_model.quant_add_1.quant_act_scale()

    block_0_int_weight_1 = quant_bottleneck_model.quant_block0_conv1.quant_weight().int(float_datatype=True)
    block_0_int_weight_2 = quant_bottleneck_model.quant_block0_conv2.quant_weight().int(float_datatype=True)
    block_0_int_weight_3 = quant_bottleneck_model.quant_block0_conv3.quant_weight().int(float_datatype=True)
    block_0_int_weight_skip = quant_bottleneck_model.shortcut.quant_weight().int(float_datatype=True)

    block_1_int_weight_1 = quant_bottleneck_model.quant_block1_conv1.quant_weight().int(float_datatype=True)
    block_1_int_weight_2 = quant_bottleneck_model.quant_block1_conv2.quant_weight().int(float_datatype=True)
    block_1_int_weight_3 = quant_bottleneck_model.quant_block1_conv3.quant_weight().int(float_datatype=True)
    
    block_0_combined_scale1=-torch.log2(init_scale*block_0_weight_scale1/block_0_relu_1) # RHS after first conv1x1 | clip 0-->255
    block_0_combined_scale2=-torch.log2(block_0_relu_1*block_0_weight_scale2/block_0_relu_2) # RHS after second conv3x3 | clip 0-->255
    block_0_combined_scale3=-torch.log2(block_0_relu_2*block_0_weight_scale3/init_scale) # RHS after third conv1x1 | clip -128-->+127
    block_0_combined_scale_skip=-torch.log2(init_scale*block_0_weight_scale_skip/init_scale) # LHS after conv1x1 | clip -128-->+127
    block_0_combined_scale4=-torch.log2(init_scale/block_0_relu_3) # After addition | clip 0-->255
    
    block_1_combined_scale1=-torch.log2(block_0_relu_3*block_1_weight_scale1/block_1_relu_1) # RHS after first conv1x1 | clip 0-->255
    block_1_combined_scale2=-torch.log2(block_1_relu_1*block_1_weight_scale2/block_1_relu_2) # RHS after second conv3x3 | clip 0-->255
    block_1_combined_scale3=-torch.log2(block_1_relu_2*block_1_weight_scale3/block_1_quant_add_1) # RHS after third conv1x1 | clip -128-->+127
    block_1_combined_scale4=-torch.log2(block_1_quant_add_1/block_1_relu_3) # After addition | clip 0-->255
    
    print("_________POST PTQ SCALES_________")
    print("init_scale:",init_scale)
    print("block_0_relu1:",block_0_relu_1)
    print("block_0_relu2:",block_0_relu_2)
    print("block_0_relu3:",block_0_relu_3)
    
    print("block_0_weight_scale1:",block_0_weight_scale1)
    print("block_0_weight_scale2:",block_0_weight_scale2)
    print("block_0_weight_scale3:",block_0_weight_scale3)
    print("block_0_weight_scale_skip:",block_0_weight_scale_skip)

    print("init_scale:",init_scale)
    print("block_1_relu1:",block_1_relu_1)
    print("block_1_relu2:",block_1_relu_2)
    print("block_1_relu3:",block_1_relu_3)
    print("--------------------------------------------------------------")
    print("block_1_weight_scale1:",block_1_weight_scale1)
    print("block_1_weight_scale2:",block_1_weight_scale2)
    print("block_1_weight_scale3:",block_1_weight_scale3)
    print("--------------------------------------------------------------")
    
    print("combined_scale block0 after first conv1x1:",block_0_combined_scale1.item())
    print("combined_scale block0 after second conv3x3:",block_0_combined_scale2.item())
    print("combined_scale block0 after third conv1x1:",block_0_combined_scale3.item())
    print("combined_scale block0 after adding skip connection:",(block_0_combined_scale4).item())
    print("combined_scale block0 after skip conv1x1:",block_0_combined_scale_skip.item())
    print("--------------------------------------------------------------")
    print("combined_scale block1 after first conv1x1:",block_1_combined_scale1.item())
    print("combined_scale block1 after second conv3x3:",block_1_combined_scale2.item())
    print("combined_scale block1 after third conv1x1:",block_1_combined_scale3.item())
    print("combined_scale block1 after adding skip connection:",(block_1_combined_scale4).item())

    q_bottleneck_out=quant_bottleneck_model(input)
    gold_out=q_bottleneck_out.int(float_datatype=True).data.numpy().astype(dtype_out)
    print("Golden::Brevitas::",gold_out)
    gold_out.tofile(log_folder+"/gold_out.txt", sep=",", format="%d")

    from brevitas.export import export_onnx_qcdq
    # ref_input = torch.ones(1, 3, 32, 32, device="cpu", dtype=dtype)
    export_onnx_qcdq(quant_bottleneck_model, input, log_folder+"/"+design+".onnx")
    # # Brevitas convolution
    q_inp = quant_id_1(input)
    int_inp = q_inp.int(float_datatype=True)
    
    before_input=int_inp.squeeze().data.numpy().astype(dtype_in)

    before_input.tofile(log_folder+"/before_ifm_mem_fmt_1x1.txt", sep=",", format="%d")
    ifm_mem_fmt = ds.reorder_mat(before_input,'YCXC8' , 'CYX' )
    ifm_mem_fmt.tofile(log_folder+"/after_ifm_mem_fmt_1x1.txt", sep=",", format="%d")
    block0_wts1 = ds.reorder_mat(block_0_int_weight_1.data.numpy().astype(dtype_wts),'OIYXI8O8' , 'OIYX' )
    block0_wts2 = ds.reorder_mat(block_0_int_weight_2.data.numpy().astype(dtype_wts),'OIYXI8O8' , 'OIYX' )
    block0_wts3 = ds.reorder_mat(block_0_int_weight_3.data.numpy().astype(dtype_wts),'OIYXI8O8' , 'OIYX' )
    block0_wts_skip = ds.reorder_mat(block_0_int_weight_skip.data.numpy().astype(dtype_wts),'OIYXI8O8' , 'OIYX' )

    total_wts=np.concatenate((block0_wts1,block0_wts2,block0_wts3,block0_wts_skip),axis=None)

    block1_wts1 = ds.reorder_mat(block_1_int_weight_1.data.numpy().astype(dtype_wts),'OIYXI8O8' , 'OIYX' )
    block1_wts2 = ds.reorder_mat(block_1_int_weight_2.data.numpy().astype(dtype_wts),'OIYXI8O8' , 'OIYX' )
    block1_wts3 = ds.reorder_mat(block_1_int_weight_3.data.numpy().astype(dtype_wts),'OIYXI8O8' , 'OIYX' )

    total_wts2=np.concatenate((total_wts,block1_wts1,block1_wts2,block1_wts3),axis=None)

    total_wts2.tofile(log_folder+"/weights_mem_fmt_final.txt", sep=",", format="%d")
    print("total_wts", total_wts2.shape)
    for i in range (0,2):
        app.buffers[2].write(ifm_mem_fmt)# input's standard format CYX | scalar YCX
        app.buffers[3].write(total_wts2) # wts's standard format OIYX | scalar OIYX
        # app.buffers[3].write(int_weight2.data.numpy().astype(dtype_in),offset=2048) # wts's standard format OIYX | scalar OIYX
        app.run()
        output3= app.buffers[4].read()
        if enable_trace:
            output3, trace = extract_trace(output3, shape_out, dtype_out)
            write_out_trace(trace, trace_file)
    # temp_out=output3.reshape(32,256, 32)
    # ofm_mem_fmt = temp_out.swapaxes(0,1)
    temp_out    = output3.reshape(32,32,32,8)
    temp2_out   = ds.reorder_mat( temp_out, 'CDYX','YCXD' )
    ofm_mem_fmt = temp2_out.reshape(256,32,32)   
    ofm_mem_fmt.tofile(log_folder+"/after_ofm_mem_fmt_final.txt", sep=",", format="%d")

    ofm_mem_fmt=torch.from_numpy(ofm_mem_fmt).unsqueeze(0)
    print("AIE output:::",ofm_mem_fmt)
    print(type(ofm_mem_fmt))
    print(type(q_bottleneck_out))
    print("difference::",torch.max(torch.abs(ofm_mem_fmt - gold_out)))

    print("difference::",torch.max(torch.abs(ofm_mem_fmt*block_1_relu_3 - q_bottleneck_out)))
    diff = torch.abs(ofm_mem_fmt - gold_out)
    print("diff::",diff)
    for i, x1 in enumerate(diff):
        for j, x2 in enumerate(x1):
            for k, x3 in enumerate(x2):
                for l, x4 in enumerate(x3):
                    if x4 > 0:
                        print("i:",i,", j:",j,", k:", k, ", l:", l, ", val:",x4)
                        print("ofm_mem_fmt val:",ofm_mem_fmt[i,j,k,l])
                        print("gold_out val:",gold_out[i,j,k,l])
    sq_abs = torch.square(torch.abs(ofm_mem_fmt - gold_out))
    print("rms::",torch.sqrt(torch.sum(sq_abs)/torch.numel(sq_abs)))
    assert(np.allclose(ofm_mem_fmt, gold_out, rtol=0, atol=2.))



_________POST PTQ SCALES_________
init_scale: tensor(0.0078)
block_0_relu1: tensor(0.0039)
block_0_relu2: tensor(0.0039)
block_0_relu3: tensor(0.0039)
block_0_weight_scale1: tensor(0.0010, grad_fn=<DivBackward0>)
block_0_weight_scale2: tensor(0.0005, grad_fn=<DivBackward0>)
block_0_weight_scale3: tensor(0.0010, grad_fn=<DivBackward0>)
block_0_weight_scale_skip: tensor(0.0010, grad_fn=<DivBackward0>)
init_scale: tensor(0.0078)
block_1_relu1: tensor(0.0039)
block_1_relu2: tensor(0.0039)
block_1_relu3: tensor(0.0039)
--------------------------------------------------------------
block_1_weight_scale1: tensor(0.0005, grad_fn=<DivBackward0>)
block_1_weight_scale2: tensor(0.0005, grad_fn=<DivBackward0>)
block_1_weight_scale3: tensor(0.0010, grad_fn=<DivBackward0>)
--------------------------------------------------------------
combined_scale block0 after first conv1x1: 9.0
combined_scale block0 after second conv3x3: 11.0
combined_scale block0 after third conv1x1: 11.0
combined_scale block0 af

C:\Users\gagan\AppData\Local\Programs\Python\Python39\lib\site-packages\torch\_tensor.py:1271: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\c10/core/TensorImpl.h:1791.)
  return super().rename(names)
C:\Users\gagan\AppData\Local\Programs\Python\Python39\lib\site-packages\brevitas\export\onnx\standard\manager.py:26: UserWarning: ONNX opset version set to 13, override with opset_version=
  warnings.warn(f"ONNX opset version set to {DEFAULT_OPSET}, override with {ka}=")


============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

total_wts (143360,)
AIE output::: tensor([[[[  5,   3, 139,  ...,   7,   5,   0],
          [  9,   8,   0,  ...,   0,   0,   0],
          [251,   0,   0,  ...,   0, 113,  69],
          ...,
          [  7,  63,   0,  ...,   0,   0, 119],
          [  0,  28,  90,  ..., 202,  72, 116],
          [  2, 103,  20,  ..., 111,   2,   0]],

         [[  2,   3, 171,  ...,  78,   0,  27],
          [ 66, 110,   0,  ...,  65,   0,   0],
          [121,   0,   1,  ...,   0,   0,   2],
          ...,
          [ 77,  26,  31,  ...,   0,  69, 190],
          [  0,   0,  43,  ...,   0,   1,   0],
          [  0,  36,  45,  ...,   0,   0,  50]],

         [[  0, 130,   0,  ...,   0, 220, 115],
          [ 27,   0,   0,  ...,   9, 124,  59],
          [  4,   5, 198,  ...,  31, 247,  20],
          

C:\Users\gagan\AppData\Local\Temp\ipykernel_18768\2285454011.py:288: UserWarning: Defining your `__torch_function__` as a plain method is deprecated and will be an error in future, please define it as a classmethod. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\python_arg_parser.cpp:367.)
  print("difference::",torch.max(torch.abs(ofm_mem_fmt*block_1_relu_3 - q_bottleneck_out)))


i: 0 , j: 21 , k: 11 , l: 6 , val: tensor(1, dtype=torch.uint8)
ofm_mem_fmt val: tensor(3, dtype=torch.uint8)
gold_out val: 2
i: 0 , j: 21 , k: 11 , l: 9 , val: tensor(1, dtype=torch.uint8)
ofm_mem_fmt val: tensor(100, dtype=torch.uint8)
gold_out val: 99
i: 0 , j: 21 , k: 11 , l: 11 , val: tensor(255, dtype=torch.uint8)
ofm_mem_fmt val: tensor(65, dtype=torch.uint8)
gold_out val: 66
i: 0 , j: 21 , k: 11 , l: 31 , val: tensor(1, dtype=torch.uint8)
ofm_mem_fmt val: tensor(11, dtype=torch.uint8)
gold_out val: 10
i: 0 , j: 21 , k: 12 , l: 1 , val: tensor(255, dtype=torch.uint8)
ofm_mem_fmt val: tensor(27, dtype=torch.uint8)
gold_out val: 28
i: 0 , j: 21 , k: 12 , l: 2 , val: tensor(1, dtype=torch.uint8)
ofm_mem_fmt val: tensor(14, dtype=torch.uint8)
gold_out val: 13
i: 0 , j: 21 , k: 12 , l: 3 , val: tensor(255, dtype=torch.uint8)
ofm_mem_fmt val: tensor(19, dtype=torch.uint8)
gold_out val: 20
i: 0 , j: 21 , k: 12 , l: 19 , val: tensor(255, dtype=torch.uint8)
ofm_mem_fmt val: tensor(25, dt

AssertionError: 

In [ ]:
if (enable_trace):
    print(trace)
else:
    print ("tracing not enabled")